This notebook downloads coincident ALOS SAR images USGS DSWE data for this project. The ALOS images are limited to high-resolution, RTC corrected, Fine Beam Dual pol (FBD) data that are acquired over North America. The USGS DSWE data is limited to the INterpreted layer With All Masking applied (INWAM) product, containing and obtained within +/- 1 days of a given SAR acquisition, containing less than 30% cloud cover, and at least 5% water surfaces.

The code in this notebook performs the following search:

> Query the ASF DAAC (via Vertex) for ALOS PALSAR data over the United States, filtering for RTC corrected FBD data <br>
>>**Iterate over returned ALOS scenes**<br>
>>>For a given ALOS PALSAR scene, find overlapping DSWE results within +/- 1 days of the SAR acquisition and having less than 30% cloud cover<br>

>>>If no DSWE data meets this criteria, move to the next ALOS scene<br>

>>>If overlapping DSWE data is available, verify that the overlap between the SAR scene and the DSWE data contains at least 50% non-cloud data, and at least 5% water<br> 


In [ ]:
# ASF and stac API libraries
import asf_search as asf
from pystac_client import Client
import pystac

# gis libraries
import geopandas as gpd
from shapely.geometry import shape, box
import rasterio
from rasterio.merge import merge
from rasterio.warp import transform_bounds
from rasterio.crs import CRS

# math imports
import numpy as np

# misc libraries
import datetime
from pathlib import Path
from collections import defaultdict
from dateutil.tz import tzutc
import netrc
import requests
from bs4 import BeautifulSoup
from typing import Union, Iterable
import os
import zipfile
import pandas as pd

In [ ]:
# we will look for ALOS-1 data over the United States
aoi_file = Path('../data/world_country_shapefiles/ne_110m_admin_0_countries.shp')
assert aoi_file.exists(), "Missing AOI file"
world = gpd.read_file(aoi_file)
usa_shape_wkt = world[world.SOVEREIGNT=="United States of America"].iloc[0].geometry.wkt
usa_shape_bounding_wkt = box(*world[world.SOVEREIGNT=="United States of America"].iloc[0].geometry.bounds).wkt

In [ ]:
# set up folder structure
sar_output_path = Path('../data/scenes')
sar_output_path.mkdir(exist_ok=True)

In [ ]:
def return_coordinates(c_list):
    '''
    Given UMM extent results from an asf_search entry, we parse the list and return a shape
    '''
    coordinates = []
    for c in c_list:
        coordinates.append([c['Longitude'], c['Latitude']])

    return shape({'type':'Polygon', 'coordinates':[coordinates]})

In [ ]:
def return_nearest_dswe_search(result):
    '''
    For an ALOS acquisition (returned from an ASF search), return a list of USGS DSWE (INWAM) of scene names that 
    overlap the acquisition and meet the search criteria (less than 30% cloud cover, within +/- 1 days of acquisition.
    Collate results by acquisition date and return list sorted by increasing timedelta from SAR acquisition date. 
    Return an empty list if no DSWE results meet the search criteria
    '''
    # geometry, startTime = shape(result['geometry']), result['properties']['startTime']
    geometry = return_coordinates(result['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]['Boundary']['Points'])
    startTime = result['TemporalExtent']['RangeDateTime']['BeginningDateTime']

    year, month, day = (int(x) for x in startTime.split('T')[0].split('-'))
    ref_date = datetime.datetime(year=year, month=month, day=day, tzinfo=tzutc())

    start_day = ref_date - datetime.timedelta(days=1)
    end_day = ref_date + datetime.timedelta(days=1)

    search_date_str = f"{start_day.strftime('%Y-%m-%d')}/{end_day.strftime('%Y-%m-%d')}"
    print(f"Acquisition date: {ref_date}, search range: {search_date_str}")
    usgs_stac_url = 'https://landsatlook.usgs.gov/stac-server'
    catalog = Client.open(usgs_stac_url)

    opts = {
    'intersects' : geometry,
    'collections':'landsat-c2l3-dswe',
    'datetime' : search_date_str,
    'max_items' : 100,
    'query':{
        'eo:cloud_cover':{'lt': 30},
            }
    }

    search = catalog.search(**opts)
    items = search.item_collection()
    
    # group the results together by acquisition date
    # A single ALOS acquisition may correspond to multiple DSWE files
    def collate_results(results):
        collated_dict = defaultdict(list)
        for r in results:
            if r.assets['inwam'].href not in collated_dict[r.datetime]:
                collated_dict[r.datetime].append(r.assets['inwam'].href)

        return collated_dict
    
    # Sort by timedelta and return the nearest result (before or after reference date)
    items = collate_results(items)

    if len(items) > 0:
        sorted_keys = sorted(items.keys(), key=lambda x:abs((x-ref_date).days))   
        return items[sorted_keys[0]]
    else:
        return []

In [ ]:
# function to download a USGS asset
def download_asset(item:Union[str, Iterable[str]], download_path:str='.'):

    if type(item) is not list : item = [item]

    download_path = Path(download_path)
    download_path.mkdir(exist_ok = True)
    
    creds = netrc.netrc()
    user,account,password = creds.authenticators('ers.cr.usgs.gov')

    url = 'https://ers.cr.usgs.gov/login'
    with requests.Session() as s:
        
        r = s.get(url)
        soup = BeautifulSoup(r.content, 'html.parser') 
        sval = soup.find('input', attrs={'name':'csrf'})['value']

        data = {"username": user, 
            "password": password,
            "csrf": sval}

        bf = s.post(url, data = data)

        downloaded_filepaths = []
        for _item in item:
            filename = _item.split('/')[-1]
            
            respb, count = None, 0
            
            while respb != 200 and count < 10:
                respb = s.get(_item,
                            allow_redirects=True,
                            headers = {'content-type': 'image/tiff'})

                with open(Path(download_path) / filename, 'wb') as src:
                    src.write(respb.content)

                count += 1

            downloaded_filepaths.append(Path(download_path) / filename)
    
    return downloaded_filepaths

# Function that returns % of valid pixels in a raster
def return_pixel_stats(filepaths, bounds, cloud_val=9):
    
    crs = CRS.from_epsg(4326)
    
    try:
        with rasterio.open(filepaths[0]) as ds:
            nodata = ds.profile['nodata']
            dst_crs = ds.crs
    except:
        return 0, 0


    bounds = transform_bounds(crs, dst_crs, *bounds)
    merged_raster, _ = merge(filepaths, bounds=bounds, nodata=nodata)

    valid_fraction = 1 - (np.sum(merged_raster == nodata) + np.sum(merged_raster == cloud_val))/merged_raster.size
    water_fraction = (np.sum(merged_raster == 1) + np.sum(merged_raster == 2))/merged_raster.size

    return valid_fraction, water_fraction

In [ ]:
start_time = datetime.datetime(year=2006, month=1, day=1, hour=0, minute=0, second=0)
time_delta = datetime.timedelta(days=60) # we will make 2 month searches with ASF
results = []

for _ in range(38):
    alos_opts = {
                'maxResults':2000,
                'platform':asf.PLATFORM.ALOS, 
                'instrument':asf.INSTRUMENT.PALSAR, 
                'processingLevel':asf.PRODUCT_TYPE.RTC_HIGH_RES,
                'polarization':asf.POLARIZATION.HH_HV, 
                'beamMode':asf.BEAMMODE.FBD, 
                'intersectsWith':usa_shape_wkt,
                'start':start_time.strftime("%Y-%m-%d"),
                'end':(start_time+time_delta).strftime("%Y-%m-%d")
                
            }

    start_time += time_delta
    pages = asf.search_generator(**alos_opts)

    for page in pages:
        results.extend([product.umm for product in page])

print(f"Number of search results: {len(results)}")

30743 results previously

In [ ]:
# Track ALOS scenes that have already been downloaded 
already_downloaded = [x.name for x in sar_output_path.iterdir() if x.is_dir()]
downloaded_alos_paths = []

for i, r in enumerate(results):
    valid_data_percentage, water_data_percentage = 0, 0
    filename = r['RelatedUrls'][0]['URL'].split('/')[-1]
    if r['RelatedUrls'][0]['URL'].split('/')[-1][:-4] in already_downloaded:
        continue

    r_shape = return_coordinates(r['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]['Boundary']['Points'])
    
    dswe_results = return_nearest_dswe_search(r)
    if len(dswe_results) == 0: # if there are no intersecting DSWE results
        print(f"no tiles for r[{i}]")
        continue
    else:
        # Download ASF file and unzip, download DSWE data
        filepaths = download_asset(dswe_results)
        valid_data_percentage, water_data_percentage = return_pixel_stats(filepaths, shape(r_shape).bounds)
        
        # Download ALOS acquisition zip file and extract
        if (valid_data_percentage >= 0.5) and (water_data_percentage >= 0.05):
            # r.download(sar_output_path)
            to_download = asf.granule_search(r['GranuleUR'])[0]
            to_download.download(sar_output_path)
            _downloaded_file = sar_output_path/filename
            assert _downloaded_file.exists(), 'Error, file does not exist'
            with zipfile.ZipFile(_downloaded_file) as f:
                f.extractall(sar_output_path)
            
            # delete zip file
            _downloaded_file.unlink()
            
            # create subfolder for dswe data and move downloaded DSWE data to it
            alos_path = (_downloaded_file).with_suffix('')
            _usgs_folder_path = (alos_path/'usgs_dswe')
            _usgs_folder_path.mkdir()
            _ = [os.rename(str(x), _usgs_folder_path/x.name) for x in filepaths]

            downloaded_alos_paths.append(alos_path.name)
        
        else: # if DSWE does not meet requirements, delete downloaded files
            print(f"Downloaded DSWE for r[{i}] did not have enough OW")
            _ = [x.unlink() for x in filepaths]

In [ ]:
alos_dswe_dict = dict()

In [ ]:
# Let's now create a database of the downloaded ALOS and DSWE scenes so that users don't have to perform the very large search
alos_scenes = [x.name for x in list(Path('../data/scenes').glob('*')) if (x.is_dir()) and ('AP_' in x.name)]

In [ ]:
data_path = Path('../data/scenes/')
for alos_scene in alos_scenes:
    usgs_path = data_path / alos_scene / 'usgs_dswe'
    usgs_tiles = ' '.join([x.name for x in list(usgs_path.glob('*INWAM*.TIF'))])
    alos_dswe_dict[alos_scene] = usgs_tiles
    

In [ ]:
df = pd.DataFrame({'ALOS_scene':alos_dswe_dict.keys(), 'DSWE_tiles':alos_dswe_dict.values()}, index=None)

In [ ]:
df.head()

In [ ]:
df.to_csv('../data/alos_dswe_database.csv', index=None)